# LDP algorithm implementations

This notebook is used to make an implementation for two different LDP perturbation algorithms. The algorithms are based on the paper by Cormode et al. called: Frequency Estimation under Local Differential Privacy, 

and the paper by Wang et al. called: Locally Differentially Provate protocols for Frequency Estimation

We will look at two different implementations of LDP algorithms:
- Unary Encoding (UE)
- BASIC-RAPPOR

## Plan
- Split the algorithm into two parts:
    - The encoding of the private data
    - The pertubation of private data
- Estimate the frequency from the aggregate of the perturbed data

---

## Unary Encoding
## Step 1: make data private at the client
### Encoding
The general idea behind unary encoding starts with changing the data into an array of bits. 

So for example:

Domain of v: {1,2,3,4}

v = 3 

Encoded v will be: [0,0,1,0]

In [1]:
v = 3
domain_v = 4

def encode(response, domain):
    return [1 if d+1 == response else 0 for d in range(domain)]

# Check if the encoding algorithm works with v=3:
encoded_v = encode(v, domain_v)
encoded_v

[0, 0, 1, 0]

### Perturbation
Perturbing data in UE is based on two operations:
- The 1 that indicates the value of v must be flipped with probability p
- The 0's must be flipped with probability q

We need to select `p` and `q` with `epsilon` set to our default value of 2, so that the following holds:
`p(1-q) / (1-p)q = exp(epsilon)`

So if we choose `p` to be 0.8 than solvin the equation will give `q` = 0.35

In [2]:
import numpy as np
import math

p = 0.8
q = 0.35

def perturb(encoded_response):
    return [perturb_bit(b, p, q) for b in encoded_response]

def perturb_bit(bit, _p, _q):
    sample = np.random.random() # produces number between [0,1)
    if bit == 1:
        if sample <= _p:
            return 1
        else:
            return 0
    elif bit == 0:
        if sample <= _q:
            return 1
        else: 
            return 0
        
perturb(encoded_v)

[1, 0, 1, 0]

## Step 2: aggregate data at server
Now that we have a way to privatize the data, we can send it to the server without any privact worries.

At the server we can estimate the frequency of each item in the domain.

For this we first make a summation of all the data per item in the domain.

With this we can apply the following function: `(sum_data - n * q) / (p - q)` 
From the summed data we remove the estimated amount of "false" data we put in with the perturbation function. 

In [3]:
def aggregate(responses):
    # Take the sum of all the columns (axis=0), ie. go over all the bits that represent the encoded response.
    sums = np.sum(responses, axis=0)
    n = len(responses)
    
    return [math.floor((v - n*q) / (p-q)) for v in sums]

## Try an example
Let's make some data to try out an example.

In [4]:
n = 10000
data = [encode(np.random.choice(range(domain_v), p=[0.5,0.3,0.15,0.05]) + 1, domain_v) for i in range(n)]
print("Some raw data: ",data[:5])

perturbed_data = [perturb(item) for item in data]
print("Some perturbed data: ", perturbed_data[:5])

sum_raw_data = np.sum(data, axis=0)
print("Sum of the raw data: ", sum_raw_data)

print("Estimate sum of perturbed data: ", aggregate(perturbed_data))


Some raw data:  [[1, 0, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0]]
Some perturbed data:  [[1, 0, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [1, 0, 1, 0], [1, 1, 0, 0]]
Sum of the raw data:  [4964 3030 1488  518]
Estimate sum of perturbed data:  [4831, 3042, 1531, 571]


## Basic Rappor
## Step 1:
### Encoding
Encoding is the same as unary encoding so that function can be reused.

### Perturbation
Perturbation is performed in **two** steps:
### RAPPOR Step 1:
Use a variable `f` to construct the first perturbation step following this formula:

`if B[i] = 1 : 1 - 1/2f`

`if B[i] = 0 : 1/2f`

The randomization of symmetric. So the probability that a 1 stays a 1 is equal to a 0 staying a zero.

This step is done **once** for each value `v` that the user has.

In [5]:
def rappor_step_one(encoded_response, f_value=0.5):
    p = 1 - (0.5 * f_value)
    q = 0.5 * f_value
    return [perturb_bit(b, p, q) for b in encoded_response]
        
rappor_step_one(encode(3,4))

[0, 0, 1, 0]

### RAPPOR Step 2:
The same step is done but now with `p` and `q` set to different values. The randomization is **symmetric** again because here `p + q = 1`

This step is done **every** time a user reports the value. 

In [6]:
def rappor_step_two(encoded_response, _p):
    p = _p
    q = 1 - p
    return [perturb_bit(b, p, q) for b in encoded_response]

rappor_step_two(rappor_step_one(encode(3,4)), 0.75)

[1, 1, 1, 0]

## Step 2:
### Aggregation
First we count the number of vectors that have their i'th bit set to 1. 
Then we correct this number for randomization.

In my example I follow the paper that says when a user does not re-report it's value, only the rappor_step_one should be used.

In [7]:
def aggregate_rappor(responses, f_value):
    sums = np.sum(responses, axis=0)
    n = len(responses)
    return [(v - (0.5 * f_value * n)) / (1 - f_value) for v in sums]

perturbed_data = [rappor_step_one(item) for item in data]

print("Some perturbed data: ", perturbed_data[:5])
print("Sum of the raw data: ", sum_raw_data)
print("Estimate sum of perturbed data: ", aggregate_rappor(perturbed_data, 0.5))

Some perturbed data:  [[1, 0, 0, 1], [0, 0, 0, 0], [0, 1, 1, 1], [0, 1, 0, 0], [0, 1, 0, 0]]
Sum of the raw data:  [4964 3030 1488  518]
Estimate sum of perturbed data:  [4916.0, 3160.0, 1296.0, 472.0]
